# Feature Importance Permutation Testing Visualization and Analysis


## Imports

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image
import os
import numpy as np
import pandas as pd
import datetime as dt

## Read in Data

In [9]:
# fi_df = pd.read_csv("FeatureImportance_Results_indexed.csv")
fi_df = pd.read_csv("network permutation t test results summary.csv")
fi_df = fi_df[~pd.isna(fi_df['Pred'])]
fi_df#['Network Subset'].unique()

,Network Subset,Mean,SD,r-Mean,r_SD,statistic,p.value,Cohens_d,parameter,conf.int,...,stderr,metric,True,Pred,n_perm,CI,CI_low,CI_high,p_past_threshold,0.000793651
14,Limbic,0.505623,0.093794,0.508320,0.091618,-0.650525,5.154279e-01,-0.029092,1996.899811,-0.010829,...,0.004146,SCZ.PredictedSCZ_percent,SCZ,SCZ,2322,0.95,-0.116746,0.058568,NaN,NaN
15,Salience / Ventral Attention,0.483662,0.083274,0.500910,0.095857,4.295475,1.828470e-05,-0.192099,1959.699209,0.025123,...,0.004015,SCZ.PredictedSCZ_percent,SCZ,SCZ,4147,0.95,-0.279930,-0.104221,*,NaN
16,Dorsal Attention,0.509410,0.084159,0.498789,0.097222,2.611905,9.072748e-03,0.116808,1957.798605,0.002646,...,0.004066,SCZ.PredictedSCZ_percent,SCZ,SCZ,4849,0.95,0.029066,0.204520,NaN,NaN
17,Visual,0.484329,0.084344,0.496360,0.097274,2.955085,3.163143e-03,-0.132155,1958.686329,0.020016,...,0.004071,SCZ.PredictedSCZ_percent,SCZ,SCZ,5365,0.95,-0.219887,-0.044391,NaN,NaN
18,Control,0.510394,0.096929,0.493269,0.098505,3.918637,9.204900e-05,0.175247,1997.480608,0.008555,...,0.004370,SCZ.PredictedSCZ_percent,SCZ,SCZ,5535,0.95,0.087404,0.263045,*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Dorsal Attention,0.354512,0.125068,0.281988,0.132205,12.602052,4.321970e-35,0.563581,1991.877812,0.061238,...,0.005755,ADHD.PredictedADHD_percent,ADHD,ADHD,4849,0.95,0.474136,0.652890,*,NaN
73,Visual,0.357874,0.121691,0.273408,0.133085,-14.811595,3.716950e-47,0.662395,1982.205317,-0.073282,...,0.005703,ADHD.PredictedADHD_percent,ADHD,ADHD,5365,0.95,0.572290,0.752342,*,NaN
74,Control,0.335759,0.119518,0.272061,0.133139,11.258494,1.525630e-28,0.503495,1975.169280,0.052602,...,0.005658,ADHD.PredictedADHD_percent,ADHD,ADHD,5535,0.95,0.414403,0.592465,*,NaN
75,Somatomotor,0.366670,0.122027,0.258663,0.133771,-18.863035,3.997040e-73,0.843581,1981.365342,-0.096778,...,0.005726,ADHD.PredictedADHD_percent,ADHD,ADHD,6370,0.95,0.753379,0.934954,*,NaN


## SetUp Color Mappings

In [10]:
size_index_df = fi_df[["Network Subset","n_perm"]].drop_duplicates(keep="first")
size_index_df.reset_index(inplace=True,drop=True)
list(size_index_df["Network Subset"])

['Limbic',
 'Salience / Ventral Attention',
 'Dorsal Attention',
 'Visual',
 'Control',
 'Somatomotor',
 'Default']

In [11]:

n_colors = len(size_index_df)#len(fi_df['Network Subset'].unique())
colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])
colors

['rgb(48, 18, 59)',
 'rgb(66, 132, 242)',
 'rgb(33, 226, 181)',
 'rgb(164, 252, 59)',
 'rgb(247, 184, 54)',
 'rgb(226, 70, 11)',
 'rgb(122, 4, 2)']

In [12]:


target_dict = {}
color_ind = 1
color_scale = {}
for index, row in size_index_df.iterrows():
  # print(color_ind)
  target_dict[row['Network Subset']] = {
    'Network Subset':row['Network Subset'],
    'Number Permuted':row['n_perm'],
    'color':colors[color_ind-1]
  }
  color_ind+=1

print(target_dict)
color_df = pd.DataFrame.from_dict(target_dict, orient='index')
fi_df = pd.merge(fi_df, color_df, how='left',on='Network Subset')
fi_df

{'Limbic': {'Network Subset': 'Limbic', 'Number Permuted': 2322, 'color': 'rgb(48, 18, 59)'}, 'Salience / Ventral Attention': {'Network Subset': 'Salience / Ventral Attention', 'Number Permuted': 4147, 'color': 'rgb(66, 132, 242)'}, 'Dorsal Attention': {'Network Subset': 'Dorsal Attention', 'Number Permuted': 4849, 'color': 'rgb(33, 226, 181)'}, 'Visual': {'Network Subset': 'Visual', 'Number Permuted': 5365, 'color': 'rgb(164, 252, 59)'}, 'Control': {'Network Subset': 'Control', 'Number Permuted': 5535, 'color': 'rgb(247, 184, 54)'}, 'Somatomotor': {'Network Subset': 'Somatomotor', 'Number Permuted': 6370, 'color': 'rgb(226, 70, 11)'}, 'Default': {'Network Subset': 'Default', 'Number Permuted': 8119, 'color': 'rgb(122, 4, 2)'}}


,Network Subset,Mean,SD,r-Mean,r_SD,statistic,p.value,Cohens_d,parameter,conf.int,...,True,Pred,n_perm,CI,CI_low,CI_high,p_past_threshold,0.000793651,Number Permuted,color
0,Limbic,0.505623,0.093794,0.508320,0.091618,-0.650525,5.154279e-01,-0.029092,1996.899811,-0.010829,...,SCZ,SCZ,2322,0.95,-0.116746,0.058568,NaN,NaN,2322,"rgb(48, 18, 59)"
1,Salience / Ventral Attention,0.483662,0.083274,0.500910,0.095857,4.295475,1.828470e-05,-0.192099,1959.699209,0.025123,...,SCZ,SCZ,4147,0.95,-0.279930,-0.104221,*,NaN,4147,"rgb(66, 132, 242)"
2,Dorsal Attention,0.509410,0.084159,0.498789,0.097222,2.611905,9.072748e-03,0.116808,1957.798605,0.002646,...,SCZ,SCZ,4849,0.95,0.029066,0.204520,NaN,NaN,4849,"rgb(33, 226, 181)"
3,Visual,0.484329,0.084344,0.496360,0.097274,2.955085,3.163143e-03,-0.132155,1958.686329,0.020016,...,SCZ,SCZ,5365,0.95,-0.219887,-0.044391,NaN,NaN,5365,"rgb(164, 252, 59)"
4,Control,0.510394,0.096929,0.493269,0.098505,3.918637,9.204900e-05,0.175247,1997.480608,0.008555,...,SCZ,SCZ,5535,0.95,0.087404,0.263045,*,NaN,5535,"rgb(247, 184, 54)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Dorsal Attention,0.354512,0.125068,0.281988,0.132205,12.602052,4.321970e-35,0.563581,1991.877812,0.061238,...,ADHD,ADHD,4849,0.95,0.474136,0.652890,*,NaN,4849,"rgb(33, 226, 181)"
59,Visual,0.357874,0.121691,0.273408,0.133085,-14.811595,3.716950e-47,0.662395,1982.205317,-0.073282,...,ADHD,ADHD,5365,0.95,0.572290,0.752342,*,NaN,5365,"rgb(164, 252, 59)"
60,Control,0.335759,0.119518,0.272061,0.133139,11.258494,1.525630e-28,0.503495,1975.169280,0.052602,...,ADHD,ADHD,5535,0.95,0.414403,0.592465,*,NaN,5535,"rgb(247, 184, 54)"
61,Somatomotor,0.366670,0.122027,0.258663,0.133771,-18.863035,3.997040e-73,0.843581,1981.365342,-0.096778,...,ADHD,ADHD,6370,0.95,0.753379,0.934954,*,NaN,6370,"rgb(226, 70, 11)"


In [13]:

fig = go.Figure()
# for i, c in enumerate(greys15):
for i, c in enumerate(colors):
  # print(i,c)
  fig.add_bar(x=[i], y = [15], marker_color = c, showlegend = False, name=c)
f = fig.full_figure_for_development(warn=False)
fig.show()
len(list(target_dict.keys()))

7

In [23]:
# outcomes = [
#   "ADHD|PredictedADHD_percent","ADHD|PredictedBIPOLAR_percent","ADHD|PredictedSCHZ_percent",
#   "BIPOLAR|PredictedADHD_percent","BIPOLAR|PredictedBIPOLAR_percent","BIPOLAR|PredictedSCHZ_percent",
#   "SCHZ|PredictedADHD_percent","SCHZ|PredictedBIPOLAR_percent","SCHZ|PredictedSCHZ_percent"
# ]
diag_list = ['SCZ','BD','ADHD']
network_list = [
  'Limbic',
  'Salience / Ventral Attention',
  'Dorsal Attention',
  'Visual',
  'Control',
  'Somatomotor',
  'Default'
]
arr_plot = make_subplots(
  rows = 3,
  cols = 3,
  row_titles=diag_list,
  column_titles=diag_list,
  x_title='Predicted',
  y_title='True',
  horizontal_spacing=.13
)
ind = 0

for i in diag_list:
  for j in diag_list:
    outcome = f'{i}|Predicted{j}_percent'
    sub_df = fi_df[(fi_df['True']==i) & (fi_df['Pred']==j)]
    print(outcome)
    sub_df['n_perm'] = sub_df['n_perm'].astype(str)
    for net in network_list:
      net_df = sub_df[sub_df['Network Subset']==net]
      if ind==0:
        arr_plot.add_trace(
          go.Bar(
            # sub_df,
            x=net_df["Network Subset"],
            y=net_df["Cohens_d"],
            marker_color=net_df["color"],
            name=f'{net} ({target_dict[net]["Number Permuted"]})',
            text=net_df["p_past_threshold"],
            textposition="outside",
            showlegend=True
          ),
          row=diag_list.index(i)+1,
          col=diag_list.index(j)+1
        )
      else:
        arr_plot.add_trace(
          go.Bar(
            # sub_df,
            x=net_df["Network Subset"],
            y=net_df["Cohens_d"],
            marker_color=net_df["color"],
            name=f'{net} ({target_dict[net]["Number Permuted"]})',
            text=net_df["p_past_threshold"],
            textposition="outside",
            showlegend=False
          ),
          row=diag_list.index(i)+1,
          col=diag_list.index(j)+1
        )
    ind+=1

      

arr_plot.update_layout(
  # height=800, 
  # width=1000, 
  # title_text=f'Network Connection Feature Importance',
  showlegend=True,
  # legend=dict(
  #   orientation='v',
  #   yanchor="bottom",
  #   y=.5,
  #   xanchor="center",
  #   x=1.2
  # ),
  font=dict(
    # family="Times New Roman",
    size=12,
    color="White"
    ),
  template='simple_white',
  paper_bgcolor='White',
  plot_bgcolor='White'
)

arr_plot.update_yaxes(range = [-.7,1.1])
arr_plot.update_xaxes(showticklabels=False)
arr_plot.for_each_annotation(
  lambda a:  a.update(x = 0,textangle=-90, xanchor='right', xshift=-40) if a.x ==.98 else
  a.update(xshift = -60) if a.x ==0 else
  a.update(yanchor = 'bottom', y=1,yshift = 25) if a.text == "Predicted" else()
  # a.update(x = -0.07) if a.text in row_titles else()
)
arr_plot.update_layout(margin=dict(t = 60, l = 120))
arr_plot.write_html('FeatureImportance_Results_03.html')
arr_plot.update_layout(
    font=dict(
      # family="Times New Roman",
      # size=10,
      color="Black"
      ),
  )
arr_plot.write_image('FeatureImportance_Results_03.png')
arr_plot.show()

SCZ|PredictedSCZ_percent
SCZ|PredictedBD_percent
SCZ|PredictedADHD_percent
BD|PredictedSCZ_percent
BD|PredictedBD_percent
BD|PredictedADHD_percent
ADHD|PredictedSCZ_percent
ADHD|PredictedBD_percent
ADHD|PredictedADHD_percent
